In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import uproot
import pickle
import nue_booster
import importlib
importlib.reload(nue_booster)

import awkward

In [ ]:
import localSettings as ls
print(ls.ntuple_path)

In [ ]:
VARLOAD = ["selected", "nu_pdg", "backtracked_pdg","category","npi0",
           "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
           "shr_tkfit_nhits_U","shr_tkfit_nhits_V","shr_tkfit_nhits_Y",
           "trk_energy_tot", "shr_hits_tot", "ccnc",
           "hits_ratio", "n_tracks_contained", 
           "NeutrinoEnergy2",
           "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
           "shrmoliereavg","shrmoliererms",
           "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
           "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
           "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
           "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
           "trk_llr_pid_score_v", # trk-PID score
           "trk_energy_proton_v", # track energy under proton hyp
           "trk_calo_energy_y_v", # track calo energy
           "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
           "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
           "hits_y", "trk_len", "slnunhits", "slnhits", "shr_score", "trk_score",
           "trk_energy", "tksh_distance", "tksh_angle",
           "shr_energy_tot_cali", "evnunhits", "nslice",
           "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
           "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
           "pfnhits","pfnunhits",
           "mcf_pass_ccpi0","mcf_pass_ncpi0","mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"
          ]
WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune"]

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

# train samples Run3
NU3  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2.root'
NUEL3 = 'prestage_prodgenie_eLee_low_overlay_det_var_run3_mcc9.1_v08_00_00_26_cv_reco2.root'
NUEH3 = 'prestage_prodgenie_eLee_high_overlay_det_var_run3_mcc9.1_v08_00_00_26_cv_reco2.root'
NCPI0S3 = 'prodgenie_nc_pi0_uboone_overlay_reco2_v08_00_00_35_sameEXT_run3_reco2_G_reco2.root'
CCPI03 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2.root' #also a test sample: split!!!!
CCNOPIT3 = 'prodgenie_tight_filter_CCmuNoPi_mcc9_v08_00_00_35_run3_reco2_reco2.root'
NCNOPIT3 = 'prodgenie_tight_filter_NCNoPi_mcc9_v08_00_00_35_run3_reco2_reco2.root'

# test samples Run3
NUE3 = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2.root'
NCPI03  = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2.root'
CCNOPI3 = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run3_reco2_reco2.root'
NCNOPI3 = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2.root'

# train samples Run3
NU1  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root'
NUEL1 = 'prestage_prodgenie_eLee_low_overlay_det_var_run1_mcc9.1_v08_00_00_26_cv_reco2.root'
NUEH1 = 'prestage_prodgenie_eLee_high_overlay_det_var_run1_mcc9.1_v08_00_00_26_cv_reco2.root'
NCPI0S1 = 'prodgenie_nc_pi0_uboone_overlay_reco2_v08_00_00_35_sameEXT_run1_reco2_reco2.root'
CCPI0S1 = 'prodgenie_cc_pi0_uboone_overlay_supplementsameEXT_v08_00_00_26_run1_reco2.root'
CCNOPIT1 = 'prodgenie_tight_filter_CCmuNoPi_mcc9_v08_00_00_35_run1_reco2_reco2.root'
NCNOPIT1 = 'prodgenie_tight_filter_NCNoPi_mcc9_v08_00_00_35_run1_reco2_reco2.root'

# test samples Run3
NUE1 = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2.root'
NCPI01  = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2.root'
CCPI01  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2.root'
CCNOPI1 = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2.root'
NCNOPI1 = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2.root'

ntuple_path = ls.ntuple_path
#ntuple_path = '/Users/cerati/Notebooks/PELEE/root_files/1205/'

u_mc3 = uproot.open(ntuple_path+NU3)[fold][tree]
u_nuel3 = uproot.open(ntuple_path+NUEL3)[fold][tree]
u_nueh3 = uproot.open(ntuple_path+NUEH3)[fold][tree]
u_ncpi0s3 = uproot.open(ntuple_path+NCPI0S3)[fold][tree]
u_ccpi03 = uproot.open(ntuple_path+CCPI03)[fold][tree]
u_ccnopit3 = uproot.open(ntuple_path+CCNOPIT3)[fold][tree]
u_ncnopit3 = uproot.open(ntuple_path+NCNOPIT3)[fold][tree]

u_nue3 = uproot.open(ntuple_path+NUE3)[fold][tree]
u_ncpi03 = uproot.open(ntuple_path+NCPI03)[fold][tree]
u_ccnopi3 = uproot.open(ntuple_path+CCNOPI3)[fold][tree]
u_ncnopi3 = uproot.open(ntuple_path+NCNOPI3)[fold][tree]

u_mc1 = uproot.open(ntuple_path+NU1)[fold][tree]
u_nuel1 = uproot.open(ntuple_path+NUEL1)[fold][tree]
u_nueh1 = uproot.open(ntuple_path+NUEH1)[fold][tree]
u_ncpi0s1 = uproot.open(ntuple_path+NCPI0S1)[fold][tree]
u_ccpi0s1 = uproot.open(ntuple_path+CCPI0S1)[fold][tree]
u_ccnopit1 = uproot.open(ntuple_path+CCNOPIT1)[fold][tree]
u_ncnopit1 = uproot.open(ntuple_path+NCNOPIT1)[fold][tree]

u_nue1 = uproot.open(ntuple_path+NUE1)[fold][tree]
u_ncpi01 = uproot.open(ntuple_path+NCPI01)[fold][tree]
u_ccpi01 = uproot.open(ntuple_path+CCPI01)[fold][tree]
u_ccnopi1 = uproot.open(ntuple_path+CCNOPI1)[fold][tree]
u_ncnopi1 = uproot.open(ntuple_path+NCNOPI1)[fold][tree]

uproot_v = [u_mc3,u_nue3,u_nuel3,u_nueh3,u_ncpi0s3,u_ccpi03,u_ccnopit3,u_ncnopit3,u_ncpi03,u_ccnopi3,u_ncnopi3,
            u_mc1,u_nue1,u_nuel1,u_nueh1,u_ncpi0s1,u_ccpi0s1,u_ccnopit1,u_ncnopit1,u_ncpi01,u_ccpi01,u_ccnopi1,u_ncnopi1]
    
mc3 = u_mc3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
nuel3 = u_nuel3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
nueh3 = u_nueh3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncpi0s3 = u_ncpi0s3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccpi03 = u_ccpi03.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccnopit3 = u_ccnopit3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncnopit3 = u_ncnopit3.pandas.df(VARLOAD + WEIGHTS, flatten=False)

nue3 = u_nue3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncpi03 = u_ncpi03.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccnopi3 = u_ccnopi3.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncnopi3 = u_ncnopi3.pandas.df(VARLOAD + WEIGHTS, flatten=False)

mc1 = u_mc1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
nuel1 = u_nuel1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
nueh1 = u_nueh1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncpi0s1 = u_ncpi0s1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccpi0s1 = u_ccpi0s1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccnopit1 = u_ccnopit1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncnopit1 = u_ncnopit1.pandas.df(VARLOAD + WEIGHTS, flatten=False)

nue1 = u_nue1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncpi01 = u_ncpi01.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccpi01 = u_ccpi01.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ccnopi1 = u_ccnopi1.pandas.df(VARLOAD + WEIGHTS, flatten=False)
ncnopi1 = u_ncnopi1.pandas.df(VARLOAD + WEIGHTS, flatten=False)

df_v = [mc3,nue3,nuel3,nueh3,ncpi0s3,ccpi03,ccnopit3,ncnopit3,ncpi03,ccnopi3,ncnopi3,
        mc1,nue1,nuel1,nueh1,ncpi0s1,ccpi0s1,ccnopit1,ncnopit1,ncpi01,ccpi01,ccnopi1,ncnopi1]

In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['protonenergy'] = trk_energy_proton_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])

In [ ]:
for i,df in enumerate(df_v):
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

In [ ]:
for i,df in enumerate(df_v):
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
    df['shr_tkfit_dedx_avg'] = (df['shr_tkfit_nhits_Y']*df['shr_tkfit_dedx_Y'] + df['shr_tkfit_nhits_U']*df['shr_tkfit_dedx_U'] + df['shr_tkfit_nhits_V']*df['shr_tkfit_dedx_V'])/df['shr_tkfit_nhits_tot']

INTERCEPT = 0.0
SLOPE = 0.83
# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]

In [ ]:
for i,df in enumerate(df_v):
    df["is_signal"] = 0
    df.loc[ df["category"] == 10, 'is_signal' ] = 1
    df.loc[ df["category"] == 11, 'is_signal' ] = 1

In [ ]:
for i,df in enumerate(df_v):
    df["train_weight"] = 1

# override here train_weight for specific samples, if needed

# set train_weight based on reco_e binning
reco_bins = np.linspace(0.15,1.55,15)
reco_scaling = [5,5,3,3,2,2,2,1,1,1,1,1,1,1]
for i,df in enumerate(df_v):
    for i, reco_bin in enumerate(reco_bins):
        if i == 0: continue
        df.loc[(df['reco_e'] > reco_bins[i-1]) & (df['reco_e'] < reco_bins[i]), 'train_weight'] = df['train_weight']*reco_scaling[i-1]

In [ ]:
MCQUERY = '((abs(nu_pdg)==12 & ccnc==0) | mcf_pass_ccpi0==1 | mcf_pass_ncpi0==1 | mcf_pass_ccnopi==1 | mcf_pass_ncnopi==1 | mcf_pass_cccpi==1 | mcf_pass_nccpi==1)'
test_mc3 = mc3.query('~'+MCQUERY)
test_mc1 = mc1.query('~'+MCQUERY)

mc3 = mc3.query(MCQUERY)
mc1 = mc1.query(MCQUERY)

train_ccpi03, test_ccpi03 = train_test_split(ccpi03, test_size=0.5, random_state=1990)

In [ ]:
# variables to be trained on
TRAINVAR = [#"n_showers_contained",
            "shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_avg","trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "is_signal","train_weight","nu_e"
        ]

In [ ]:
print (ls.pickle_path)

In [ ]:
train_mc = pd.concat([mc3,mc1])
train_nue = pd.concat([nuel3,nueh3,nuel1,nueh1])
train_ncpi0 = pd.concat([ncpi0s3,ncpi0s1])
train_ccpi0 = pd.concat([train_ccpi03,ccpi0s1])
train_ccnopi = pd.concat([ccnopit3,ccnopit1])
train_ncnopi = pd.concat([ncnopit3,ncnopit1])

test_mc = pd.concat([test_mc3,test_mc1])
test_nue = pd.concat([nue3,nue1])
test_ncpi0 = pd.concat([ncpi03,ncpi01])
test_ccpi0 = pd.concat([test_ccpi03,ccpi01])
test_ccnopi = pd.concat([ccnopi3,ccnopi1])
test_ncnopi = pd.concat([ncnopi3,ncnopi1])

samples = {
    "mc": (train_mc, test_mc),
    "nue": (train_nue, test_nue),
    "ncpi0": (train_ncpi0, test_ncpi0),
    "ccpi0": (train_ccpi0, test_ccpi0),
    "ccnopi": (train_ccnopi, test_ccnopi),
    "ncnopi": (train_ncnopi, test_ncnopi)
} 

fig, ax = plt.subplots(1,1)

my_booster = nue_booster.NueBooster(samples, TRAINVAR, random_state=1990)

print (my_booster.variables)

PRESEL = "reco_e < 1.2"
PRESEL += ' and nslice == 1'
PRESEL += ' and selected == 1'
PRESEL += ' and shr_energy_tot_cali > 0.07'
PRESEL += ' and n_tracks_contained > 0'
PRESEL += ' and CosmicIPAll3D > 20.'
PRESEL += ' and trkpid < 0.1'
PRESEL += ' and hits_ratio > 0.5'
PRESEL += ' and shr_score < 0.40'
PRESEL += ' and n_showers_contained == 1'
PRESEL += ' and tksh_distance < 6.0'
PRESEL += ' and shr_tkfit_dedx_avg < 4.0'
PRESEL += ' and tksh_angle > -0.9'

my_booster.set_preselection(PRESEL)

gain_imp = {}

for label, bkg_query in zip(nue_booster.labels, nue_booster.bkg_queries):
    
    preds, gain_imp[label] = my_booster.train_booster(ax, bkg_query)
    
    with open(ls.pickle_path+'booster_%s_test.pickle' % label, 'wb') as booster_file:
        pickle.dump(preds, booster_file)

    #variables = my_booster.variables.copy()
    #print ('variables are : ',variables)
    #variables.remove("is_signal")
    #variables.remove("nu_e")
    #variables.remove("train_weight")        
        
    #mc_prediction = preds.predict(
    #    xgb.DMatrix(mc[variables]),
    #    ntree_limit=preds.best_iteration)
    #nue_prediction = preds.predict(
    #    xgb.DMatrix(nue[variables]),
    #    ntree_limit=preds.best_iteration)
    #ext_prediction = preds.predict(
    #    xgb.DMatrix(ext[variables]),
    #    ntree_limit=preds.best_iteration)
    #ncpi0_prediction = preds.predict(
    #    xgb.DMatrix(ncpi0[variables]),
    #    ntree_limit=preds.best_iteration)
    #ccpi0_prediction = preds.predict(
    #    xgb.DMatrix(ccpi0[variables]),
    #    ntree_limit=preds.best_iteration)

    #ncpi0["%s_score" % label] = ncpi0_prediction
    #ccpi0["%s_score" % label] = ccpi0_prediction
    #mc["%s_score" % label] = mc_prediction
    #nue["%s_score" % label] = nue_prediction
    #ext["%s_score" % label] = ext_prediction

#ax.set_ylim([0, 1.05])
#ax.set_xlim([0, 1.0])
#ax.set_xlabel('False Positive Rate')
#ax.set_ylabel('True Positive Rate')
#ax.set_title('ROC')
#ax.legend()
#fig.tight_layout()
#fig.savefig(ls.plots_path+"roc_single.pdf")

In [ ]:
print(gain_imp['pi0'])
print(gain_imp['nonpi0'])

if 'shr_tkfit_gap10_dedx_V' in gain_imp['pi0']:
    gain_imp['pi0']['shr_tkfit_gap10_dedx_ALL'] = gain_imp['pi0']['shr_tkfit_gap10_dedx_V']+gain_imp['pi0']['shr_tkfit_gap10_dedx_U']+gain_imp['pi0']['shr_tkfit_gap10_dedx_Y']
    gain_imp['nonpi0']['shr_tkfit_gap10_dedx_ALL'] = gain_imp['nonpi0']['shr_tkfit_gap10_dedx_V']+gain_imp['nonpi0']['shr_tkfit_gap10_dedx_U']+gain_imp['nonpi0']['shr_tkfit_gap10_dedx_Y']
    del gain_imp['pi0']['shr_tkfit_gap10_dedx_V']
    del gain_imp['pi0']['shr_tkfit_gap10_dedx_U']
    del gain_imp['pi0']['shr_tkfit_gap10_dedx_Y']
    del gain_imp['nonpi0']['shr_tkfit_gap10_dedx_V']
    del gain_imp['nonpi0']['shr_tkfit_gap10_dedx_U']
    del gain_imp['nonpi0']['shr_tkfit_gap10_dedx_Y']
if 'shr_tkfit_2cm_dedx_V' in gain_imp['pi0']:
    gain_imp['pi0']['shr_tkfit_2cm_dedx_ALL'] = gain_imp['pi0']['shr_tkfit_2cm_dedx_V']+gain_imp['pi0']['shr_tkfit_2cm_dedx_U']+gain_imp['pi0']['shr_tkfit_2cm_dedx_Y']
    gain_imp['nonpi0']['shr_tkfit_2cm_dedx_ALL'] = gain_imp['nonpi0']['shr_tkfit_2cm_dedx_V']+gain_imp['nonpi0']['shr_tkfit_2cm_dedx_U']+gain_imp['nonpi0']['shr_tkfit_2cm_dedx_Y']
    del gain_imp['pi0']['shr_tkfit_2cm_dedx_V']
    del gain_imp['pi0']['shr_tkfit_2cm_dedx_U']
    del gain_imp['pi0']['shr_tkfit_2cm_dedx_Y']
    del gain_imp['nonpi0']['shr_tkfit_2cm_dedx_V']
    del gain_imp['nonpi0']['shr_tkfit_2cm_dedx_U']
    del gain_imp['nonpi0']['shr_tkfit_2cm_dedx_Y']

labels = []
pi0_imp = []
nonpi0_imp = []

for i in sorted (gain_imp['pi0'].keys()) :  
    labels.append(i)
    pi0_imp.append(gain_imp['pi0'][i])
    nonpi0_imp.append(gain_imp['nonpi0'][i])
    
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(8,6))
rects1 = ax.bar(x - width/2, pi0_imp, width, label='pi0')
rects2 = ax.bar(x + width/2, nonpi0_imp, width, label='nonpi0')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('total gain')
ax.set_title('BDT Variable Importance')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation = 90)
ax.legend()

plt.show()
fig.tight_layout()
#fig.savefig(ls.plots_path+"0109/bdt_var_gain.pdf")

In [ ]:
d_pi0_rnk = {}
rnk = len(labels)
for w in sorted(gain_imp['pi0'], key=gain_imp['pi0'].get, reverse=True):
    d_pi0_rnk[w] = rnk
    rnk = rnk-1
print(d_pi0_rnk)

d_nonpi0_rnk = {}
rnk = len(labels)
for w in sorted(gain_imp['nonpi0'], key=gain_imp['nonpi0'].get, reverse=True):
    d_nonpi0_rnk[w] = rnk
    rnk = rnk-1
print(d_nonpi0_rnk)

pi0_rnk = []
nonpi0_rnk = []

for i in labels:  
    pi0_rnk.append(d_pi0_rnk[i])
    nonpi0_rnk.append(d_nonpi0_rnk[i])
    
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(8,6))
rects1 = ax.bar(x - width/2, pi0_rnk, width, label='pi0')
rects2 = ax.bar(x + width/2, nonpi0_rnk, width, label='nonpi0')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Rank (based on total gain)')
ax.set_title('BDT Variable Importance')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation = 90)
ax.legend()

plt.show()
fig.tight_layout()
#fig.savefig(ls.plots_path+"0109/bdt_var_rank.pdf")